In [1]:
import numpy as np
import pandas as pd
from bokeh.layouts import row, widgetbox
from bokeh.models import CustomJS, Slider, Select
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import push_notebook, output_notebook, curdoc
from bokeh.client import push_session
output_notebook()

#create sample pandaframe to work with, this will store the actual data
a = np.arange(50).reshape((5,10))
labels = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J"]
val_a = pd.DataFrame(a, columns=labels )

# Here is a dict of some keys that I want to be able to pick from for plotting
axis_map = {
    "A": "A",
    "B": "B",
    "C": "C"
}

#This is to update during the callback
code = ''' var data = source.data;
           var value1 = val1.value;
           var value2 = val2.value;
           var original_data = original_source.data
           // get data corresponding to selection
           x = original_data[value1];
           y = original_data[value2];
           data['x'] = x;
           data['y'] = y;
           source.trigger('change');
           // set axis labels
           x_axis.axis_label = value1
           y_axis.axis_label = value2
            '''
source = ColumnDataSource(data=dict(x=val_a['A'], y=val_a['B']))
original_source = ColumnDataSource(data=val_a.to_dict(orient='list'))


#plot the figures
plot = figure(plot_width=400, plot_height=400)
plot.circle(x= "x",y="y", source=source, line_width=3, line_alpha=0.6)


callback = CustomJS(args=dict(source=source, original_source = original_source, x_axis=plot.xaxis[0],y_axis=plot.yaxis[0]), code=code)

#Create two select widgets to pick the features of interest 
x_axis = Select(title="X Axis", options=sorted(axis_map.keys()), value="A", callback = callback)
callback.args["val1"] = x_axis

y_axis = Select(title="Y Axis", options=sorted(axis_map.keys()), value="B", callback = callback)
callback.args["val2"] = y_axis

plot.xaxis[0].axis_label = 'A'
plot.yaxis[0].axis_label = 'B'

#Display the graph in a jupyter notebook
layout = row(plot, x_axis, y_axis)
show(layout, notebook_handle=True)

Loading BokehJS ...

In [3]:
from bokeh.models import  Callback, ColumnDataSource, Rect, Select,CustomJS
from bokeh.plotting import figure, output_file, show,  gridplot
from bokeh.models.layouts import VBox,HBox
import numpy as np


#Color Dictionary
redcolor5 = {u'All ages': "#720017", u'70+ years': "#bd0026", u'50-69 years':  "#f03b20", u'15-49 years': "#fd8d3c", u'Under 5 years': "#f4cc63", u'gridline': '#b2ada6', u'background': '#e3e0db', u'axis' : '#aba9a7'}

#Just a sample of my data
country_both = ['China', 'India', 'United States', 'Russia', 'Japan', 'Indonesia', 'Germany', 'United Kingdom', 'Italy', 'Brazil']
ages_gen = ['Under 5 years', '15-49 years', '50-69 years', '70+ years', 'All ages', 'Age-standardized']

height70yr = np.array([919470, 421922, 321125, 193960, 148946, 107822, 97529, 90198, 81107, 76782])
height50to69 = np.array([640496, 626995, 182338, 195472, 40422, 109242, 44161, 33333, 24964, 64429])
height15to49 = np.array([126094, 139420, 26159, 43239, 5480, 39040, 6829, 4163, 3571, 16152])
heightUnder5 = np.array([10210, 43338, 82, 714, 41, 5255, 0, 26, 0, 1201])
zeros = np.zeros(len(country_both))


#Y Values for Stacked bar chart
yUnder5 = heightUnder5 / 2.0
y15to49 = yUnder5 + height15to49/2.0
y50to69 = y15to49 + height50to69/2.0
y70yr = y50to69 + height70yr/2.0


output_file('UW_TobaccoDeath.html')

#Figure for Stacked bar chart
p1 = figure(title="Top Countries with Death Due to Tobacco by Age", 
            x_range=country_both, y_range=[0, np.amax([y70yr+height70yr])], 
            plot_width=700, plot_height = 600,
            outline_line_color= None)


#source for callback
source1 = ColumnDataSource(data=dict(x=country_both, y = yUnder5, y_full = yUnder5, height = heightUnder5, height_full = heightUnder5 ,height_zeros = zeros, y_zeros = heightUnder5 / 2.0))
source2 = ColumnDataSource(data=dict(x=country_both, y = y15to49, y_full = y15to49, height = height15to49, height_full = height15to49,height_zeros = zeros, y_zeros = height15to49 / 2.0))
source3 = ColumnDataSource(data=dict(x=country_both, y = y50to69, y_full = y50to69, height = height50to69, height_full = height50to69,height_zeros = zeros , y_zeros = height50to69 / 2.0))
source4 = ColumnDataSource(data=dict(x=country_both, y = y70yr, y_full = y70yr, height = height70yr, height_full = height70yr,height_zeros = zeros, y_zeros =  height70yr / 2.0))

#Use rect glyphs for stached bars
p1.rect(x ='x', y ='y', width =.8, height = 'height', source = source1, color=redcolor5['Under 5 years'], alpha=0.8, name = "Under 5")
p1.rect(x = 'x', y ='y', width = .8, height ='height', source = source2, color=redcolor5['15-49 years'], alpha=0.8, name = "15 to 49")
p1.rect(x = 'x', y ='y', width = .8, height ='height', source = source3, color=redcolor5['50-69 years'], alpha = .8, name = "50 to 69")
p1.rect(x = 'x', y ='y', width = .8, height ='height', source = source4, color=redcolor5['70+ years'], alpha = .8, name = "70+ yrs")

#Java script Callbacks for age  
#I want this to recognize the 70+ year old drop down selection 
#and change the plot so that the height of the glyph is the same as the y value and the 70 year old glyph is the only one that displays
Callback_Age = CustomJS(args={'source1': source1,'source2': source2,'source3': source3,'source4': source4}, code="""
        var f = cb_obj.get('value');
        var data1 = source1.get('data');
        var data2 = source2.get('data');
        var data3 = source3.get('data');
        var data4 = source4.get('data');
        if (f == 'Under 5 years') {
            data3['height'] = data3['height_zeros'];
            data2['height'] = data2['height_zeros'];
            data4['height'] = data4['height_zeros'];
            data1['y'] = data1['y_zeros'];
            data1['height'] = data1['height_full'];
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }
        if (f == '15-49 years') {
            data1['height'] = data1['height_zeros'];
            data3['height'] = data3['height_zeros'];
            data4['height'] = data4['height_zeros'];
            data2['y'] = data2['y_zeros'];
            data2['height'] = data2['height_full'];
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }

        if (f == '50-69 years') {
            data1['height'] = data1['height_zeros'];
            data2['height'] = data2['height_zeros'];
            data4['height'] = data4['height_zeros'];
            data3['y'] = data3['y_zeros'];
            data3['height'] = data3['height_full'];
            console.log('data3',data3)
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }
        if (f == '70+ years') {
            data1['height'] = data1['height_zeros'];
            data2['height'] = data2['height_zeros'];
            data3['height'] = data3['height_zeros'];
            data4['y'] = data4['y_zeros'];
            data4['height'] = data4['height_full'];
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }
        if (f == 'All ages') {
            data1['height'] = data1['height_full'];
            data1['y'] = data1['y_full'];
            data2['height'] = data2['height_full'];
            data2['y'] = data2['y_full'];
            data3['height'] = data3['height_full'];
            data3['y'] = data3['y_full'];
            data4['height'] = data4['height_full'];
            data4['y'] = data4['y_full'];
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }



    """)
#Use the Select widget
dropdown_age = Select(title="Ages:", value=ages_gen[4], options= ages_gen,  callback = Callback_Age)

#Display data
filters = VBox(dropdown_age)
tot =  HBox(filters, gridplot([[p1]]))
show(tot)

C:\Users\db375qf\AppData\Local\Continuum\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: bokeh.models.VBox was deprecated in Bokeh 0.12.5 and will be removed, use bokeh.models.Column instead.
  warn(message)
C:\Users\db375qf\AppData\Local\Continuum\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: bokeh.models.HBox was deprecated in Bokeh 0.12.5 and will be removed, use bokeh.models.Row instead.
  warn(message)


In [1]:
from datetime import date
from random import randint
import bokeh
import bokeh.plotting 

data = dict(dates=[date(2014, 3, i+1) for i in range(10)],
            downloads=[randint(0, 100) for i in range(10)])

source = bokeh.models.ColumnDataSource(data)

columns = [bokeh.models.TableColumn(field="dates", title="Date",
           formatter=bokeh.models.DateFormatter()),
           bokeh.models.TableColumn(field="downloads", title="Downloads")]

source.callback = bokeh.models.CustomJS(args=dict(source=source), code="""
       console.log( '#Selected rows:');
       var indices = source.selected["1d"].indices;
       for (var i = 0; i<indices.length; i++){
           console.log(i+":"+indices[i]);
       }
       """)

data_table = bokeh.models.DataTable(source=source, columns=columns, 
                                    width=400, height=280, editable=True)

bokeh.io.output_notebook()
bokeh.io.show(data_table)

Loading BokehJS ...